<h2>Set up instruction<h2>
Create a folder in Google Drive COMP0087 your own google drive and inside that folder create 3 more subfolder named DataRequired,  BertModels, and lstmFinalModel.


Download the files from this link or create a shortcut to these folders.
This is link is for the data set  and paste the content into the folder DataRequired


https://drive.google.com/drive/folders/1UmIaTE_dTOSCBf-ardkr9eo6WWQ5oEiz?usp=sharing


This is link is for the bert model copy and paste the content into the folder BertModels
https://drive.google.com/drive/folders/1vcKPAlvTPQarftyocat1oF-hPQ9eZumE?usp=sharing


This is link is for the lstm models copy and paste the content into the folder lstmFinalModel


 https://drive.google.com/drive/folders/1xlKIV8s1T3FJDTxfVgpCAscr-CSJ7EXk?usp=sharing

In [ ]:
#!pip install pytorch-pretrained-bert pytorch-nlp
#from pytorch_pretrained_bert import BertConfig
!pip install transformers
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertConfig
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from google.colab import drive
import os
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import cohen_kappa_score
import itertools

drive.mount('/content/drive')
os.chdir('drive/MyDrive/COMP0087/')



     |████████████████████████████████| 2.3MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 46.6MB/s 
     |████████████████████████████████| 901kB 48.8MB/s 
Mounted at /content/drive


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

LSTM Demo

In [ ]:
import numpy as np
import pandas as pd
import nltk 
from nltk.corpus import stopwords                   
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer          
from gensim.models import Word2Vec
import re
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
import gensim
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

In [ ]:
# In case we wanna use it later
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Download w2v
%%shell
import wget

wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


/bin/bash: import: command not found
--2021-05-31 10:37:18--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.111.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.111.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  6.04MB/s    in 7m 6s   

2021-05-31 10:44:25 (3.69 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
%%shell
gunzip GoogleNews-vectors-negative300.bin.gz

gzip: GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)? n
	not overwritten


CalledProcessError: ignored

In [ ]:
model_g = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

In [ ]:
# import file 
df = pd.read_csv("DataRequired/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1')

In [ ]:
# create a list of tokens using every word in essay
# For testing now, to check if the baseline is working
def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    #words = essay_v.lower().split()
    words = essay_v.split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return words

In [ ]:
wv = model_g.wv
# Just to test, averaged all the word embeddings in the whole paragraph into a single embedding vector.
# function to change list of tokens to word embedding using word2vec
# not being used
def avgW2vRepresentation( passage ):
    counterWordsVocab = 0
    w2vRepresentation = np.zeros(300)
    for words in passage:
        if words in wv:
            counterWordsVocab +=1
            w2vRepresentation += wv[words]

    if(counterWordsVocab != 0):
        w2vRepresentation /= counterWordsVocab

    return w2vRepresentation

# is being used
def embed_paragraph(passage):
  embedded_vect = []
  for words in passage:
    if words in wv:
      embedded_vect.append(wv[words])
    
  return np.array(embedded_vect)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size = 300, hidden_layer_size_1 = 300, hidden_layer_size_2 = 64, output_size = 16, loss_function = None, lr = 0.001, dropout_amount = 0.4):
        super().__init__()
        self.hidden_layer_size_1 = hidden_layer_size_1
        self.hidden_layer_size_2 = hidden_layer_size_2
        self.lstm = nn.LSTM(input_size, hidden_layer_size_1, batch_first = True, dropout = dropout_amount) 
        self.lstm_2 = nn.LSTM(hidden_layer_size_1, hidden_layer_size_2, batch_first = True, dropout = dropout_amount)        
        self.linear = nn.Linear(hidden_layer_size_2, output_size)

        self.loss = loss_function if loss_function is not None else nn.CrossEntropyLoss()
        self.lr = lr

        return None
        

    def forward(self, input_seq):
        # print(f"Input shape {input_seq.shape}")
        # lstm_out, self.hidden_cell = self.lstm(input_seq.view(1, input_seq.shape[0], input_seq.shape[1]))
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        lstm_out, self.hidden_cell = self.lstm_2(lstm_out)
        result_unpacked, lengths_unpacked = pad_packed_sequence(lstm_out, batch_first = True)
        linear_input = result_unpacked[np.arange(result_unpacked.shape[0]), lengths_unpacked - 1, :]
        # predictions = self.linear(lstm_out.view(input_seq.shape[0], -1))
        predictions = self.linear(linear_input)
        #predictions = F.relu(predictions)
        return predictions

def pack_data(data, lengths):
  input_tens = torch.tensor(data, dtype = torch.float)
  packed_seq = pack_padded_sequence(input_tens, lengths, batch_first = True, enforce_sorted = False)
  return packed_seq

In [ ]:
models = []
for index  in range(5):
  modelLabel = "lstmFinalModel"+str(index)
  models.append(torch.load("lstmModels/"+modelLabel))
  print(modelLabel)

lstmFinalModel0
lstmFinalModel1
lstmFinalModel2
lstmFinalModel3
lstmFinalModel4


In [ ]:
#get all the essay set for that particular essay set
def import_data_for_testing(essay_set):
  df_train = pd.read_excel('DataRequired/training_set_rel3.xlsx')
  
  
  if essay_set is not None:
    df_train = df_train[df_train['essay_set']==essay_set]

  
  return df_train
def getRealScore (df):
  dfFormat = df[[df.columns[2], df.columns[6],df.columns[10], df.columns[11], df.columns[12],df.columns[13],\
                df.columns[16],df.columns[17],df.columns[18],df.columns[19] ]]
  # prepare test data
  test_data_array = np.array(dfFormat)
  #print(test_data_array)
  N_t,d_t = test_data_array.shape
  y_test = np.zeros((N_t, 5))
  #x_train[:,0] = train_data_array[:,0]
  y_test[:,0] = test_data_array[:,1]
  y_test[:,1] = test_data_array[:,2] + test_data_array[:,6]
  y_test[:,2] = test_data_array[:,3] + test_data_array[:,7]
  y_test[:,3] = test_data_array[:,4] + test_data_array[:,8]
  y_test[:,4] = test_data_array[:,5] + test_data_array[:,9]
  return y_test
def processing (test_data):
  tokenized_no_stopwords = []; finalized_essay = []
  essays = np.array(test_data['essay'])
  essay_1 = essays[0]
  for essay in essays:
    tokenized_no_stopwords.append(essay_to_wordlist(essay, True))

  max_len = 0
  for tokenized_essay in tokenized_no_stopwords:
    em_par = embed_paragraph(tokenized_essay)
    par_len = em_par.shape[0]
    if max_len < par_len:
      max_len = par_len
    finalized_essay.append(em_par)

  # Pad anyhow, and ignore with padded
  padded_embed = []; lengths = []
  for embedding in finalized_essay:
    essay_len = embedding.shape[0]
    template = np.zeros((max_len, 300))
    template[:essay_len, :] = embedding
    padded_embed.append(template)
    lengths.append(essay_len)
  test_data = pack_data(padded_embed, lengths)
  return test_data

def evaluate_model_demo(models, data):
  input = data.to(device); scores = []
  for model_idx in range(len(models)):
    model = models[model_idx]
    preds = model(input).cpu().detach().numpy()
    preds = np.argmax(preds, axis = 1).reshape(-1)
    #label = labels[:, model_idx].reshape(-1)
    # if score is 'kappa':
    #   pred_score = cohen_kappa_score(preds, label, weights = 'quadratic')
    # elif score is 'acc':
    #   pred_score = np.mean(preds == label)
    
    scores.append(preds)

  return scores

In [ ]:
#getting data for essay set 7
dfOnEssaySet = import_data_for_testing(7)

In [ ]:
#set true to get a random essay if else set random to false and input the desired essay id in the chosen id
random = False
chosenId = 18121
if(random):
  chosenId = np.random.choice(dfOnEssaySet['essay_id'])

essaydf =  dfOnEssaySet[dfOnEssaySet['essay_id'] == chosenId ]
print(essaydf)
essay = essaydf['essay'].values

print(essay[0])

       essay_id  essay_set  ... rater3_trait5  rater3_trait6
10941     18121          7  ...           NaN            NaN

[1 rows x 28 columns]
I am never paitent, but which I am it is only once and a while. Sitting down and not doing a thing drives me to be insane. When I’m doing nothing and it is really quiet. I get really bad headaches. In school for example and I’m waiting I get rigily like I tap my foot at tap my pencil and my desk. Tap ???everything things like that. I must do something or I go insane. Waiting in a doctors office, or flying on a plane for a while can get real boring. When I wait to long I feel lazy and tired because I’m not moving at all. I get buggy when I’m paitent, and waiting for something. It is usually no longer than @NUM1 min (half an hour) before I get real bad. I can’t be paitent very long or I get buggy. One way I help myself is playing sports and doing things constantive.


Predicted result and actual result from lstm

In [ ]:
predictedScore = evaluate_model_demo(models,processing(essaydf))
print("Predicted Score")
print(predictedScore)
actualScore = getRealScore(essaydf)
print("Actual Score")
print(actualScore)

Predicted Score
[array([20]), array([2]), array([4]), array([4]), array([5])]
Actual Score
[[14.  2.  2.  4.  6.]]


To get predicted result for any essay using lstm

In [ ]:
#inputting your own essay in the dictionary essay below
dictionaryEssay = {}
dictionaryEssay['essay'] = ["I am never paitent, but which I am it is only once and a while. Sitting down and not doing a thing drives me to be insane. When I’m doing nothing and it is really quiet. I get really bad headaches. In school for example and I’m waiting I get rigily like I tap my foot at tap my pencil and my desk. Tap ???everything things like that. I must do something or I go insane. Waiting in a doctors office, or flying on a plane for a while can get real boring. When I wait to long I feel lazy and tired because I’m not moving at all. I get buggy when I’m paitent, and waiting for something. It is usually no longer than @NUM1 min (half an hour) before I get real bad. I can’t be paitent very long or I get buggy. One way I help myself is playing sports and doing things constantive."]
print(dictionaryEssay)
dfown = pd.DataFrame.from_dict(dictionaryEssay)
predictedScore = evaluate_model_demo(models,processing(dfown))
print("Predicted Score")
print(predictedScore)
#For multiple model the format is the following
# overall score, trait1, trait 2, trait 3, trait 4

{'essay': ['I am never paitent, but which I am it is only once and a while. Sitting down and not doing a thing drives me to be insane. When I’m doing nothing and it is really quiet. I get really bad headaches. In school for example and I’m waiting I get rigily like I tap my foot at tap my pencil and my desk. Tap ???everything things like that. I must do something or I go insane. Waiting in a doctors office, or flying on a plane for a while can get real boring. When I wait to long I feel lazy and tired because I’m not moving at all. I get buggy when I’m paitent, and waiting for something. It is usually no longer than @NUM1 min (half an hour) before I get real bad. I can’t be paitent very long or I get buggy. One way I help myself is playing sports and doing things constantive.']}
Predicted Score
[array([20]), array([2]), array([4]), array([4]), array([5])]


Bert model Demo

In [ ]:
def tokenize_sentences(sentences, max_len):
  sentences = ["[CLS] " + query + " [SEP]" for query in sentences]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  text_len = torch.LongTensor([len(txt) for txt in tokenized_texts])
  input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
  input_ids[input_ids[:,-1]!=0,-1] = tokenizer.sep_token_id  #if a sentence has been cut short, make sure last token is SEP
  input_ids = torch.LongTensor(input_ids)

  #attention masks define which tokens the model pays attention to
  attention_masks = np.ones(input_ids.shape)
  attention_masks = torch.LongTensor(attention_masks)
  attention_masks[input_ids==0] = 0

  return input_ids, attention_masks, text_len

In [ ]:
class EssayModel(nn.Module):
  """Model for essay grading using BERT embeddings"""
  def __init__(self, hidden_dim, encode_question, regression, dropout, 
               tokenized_prompt, attention_prompt, output_shape=1):
    super().__init__()
    self.bert_dim = 768
    self.encode_question = encode_question    
    self.regression = regression
    self.output_shape = output_shape
    self.dropout = dropout
    self.tokenized_prompt = tokenized_prompt
    self.attention_prompt = attention_prompt

    if self.encode_question:
      self.hidden_dim = hidden_dim
    else:
      self.hidden_dim = self.bert_dim

    #network layers
    configuration = BertConfig()
    configuration.attention_probs_dropout_prob = dropout
    configuration.hidden_dropout_prob = dropout
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, config=configuration).to(device)    
    #print(self.bert)
    
    if self.encode_question:
      self.l1 = nn.Linear(in_features=2*self.bert_dim, out_features=self.hidden_dim)
      self.BatchNorm = nn.BatchNorm1d(self.hidden_dim)

    self.l2 = nn.Linear(in_features=self.hidden_dim, out_features=self.output_shape)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_ids, attention_masks):
    x = self.bert(input_ids, attention_masks)['pooler_output']
    if self.encode_question:
      #tokenized_prompt = tokenized_prompt.expand(x.shape[0], -1)
      prompt_encoding = self.bert(self.tokenized_prompt, self.attention_prompt)['pooler_output']
      prompt_encoding = prompt_encoding.expand(x.shape[0], -1)
      x = torch.cat([x, prompt_encoding], dim=1)
      x = F.relu(self.BatchNorm(self.l1(x)))
    
    #if a classification task, apply softmax activation
    if self.regression=='reg':
      x = self.l2(x)
    elif self.regression=='multi':
      x = self.l2(x)
      x = torch.cat([x,x.sum(1).view(-1,1)], axis=1)  #append total grade to the end
    elif self.regression=='class':
      x = self.l2(x)
    return x

Set Qns number the bert model is from


In [ ]:
qnsNumber = 2
encode_question = False
if(qnsNumber == 1):
  lr = 1e-5
  
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 2):
  lr = 1e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 3):
  lr = 1e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 4):
  lr = 1e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 5):
  lr = 1e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 6):
  lr = 1e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 7):
  lr = 2e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
  lr = 2e-5
  reg = 'multi'
  dropout= 0.1
  model_label2 = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
elif(qnsNumber == 8):
  lr = 2e-5
  reg = 'reg'
  dropout= 0.1
  model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)



In [ ]:
print(model_label)
# encode_question = False
# lr = 2e-5
# reg = 'reg'
# dropout= 0.1
# model = EssayModel(hidden_dim=512, encode_question=encode_question, 
#                        regression=reg, output_shape=output_shape, 
#                        dropout=dropout, tokenized_prompt=tokenized_prompt,
#                        attention_prompt=attention_prompt).to(device)
# model_label = 'q'+str(qnsNumber)+'_encode{0}_alpha{1}_reg{2}_dropout{3}'.format(encode_question, lr, reg, dropout)
model = torch.load("BertModels/"+model_label )
model.eval()
if (qnsNumber ==7):
  print(model_label2)
  model2 = torch.load(model_label2 )
  model2.eval()

q2_encodeFalse_alpha1e-05_regreg_dropout0.1


Randomly or Select a particular essay id from the essay set qns specified above


In [ ]:
dfOnEssaySet = import_data_for_testing(qnsNumber)
#dfOnEssaySet.head()
# dfOnEssaySet.head()
#set true to get a random essay if else set random to false and input the desired essay id in the chosen id
random = True
chosenId = 18121
if(random):
  chosenId = np.random.choice(dfOnEssaySet['essay_id'])

essaydf =  dfOnEssaySet[dfOnEssaySet['essay_id'] == chosenId ]
print(essaydf)
essay = essaydf['essay'].values

print(essay[0])

      essay_id  essay_set  ... rater3_trait5  rater3_trait6
3013      4208          2  ...           NaN            NaN

[1 rows x 28 columns]
One @MONTH1 subject the censorship of library content in certain places and times, whether fictional or not, to his or her own opinion.  I personally disagree with the concept.  The facts point out that society fears the dystopia predicted by publications exploring the outcome of similar campaigns and has instated systems designed to prevent such occurrences.  Certain examples stand in play to explicate my opinion.       Society, at least in my native @LOCATION1 and surely abroad in other democratic countries, dislikes enforced censorship for a few reasons.  First, many countries have a @CAPS1 of @CAPS2 and priviledges upheld in the face of the law.  Our founding fathers wrote out one of the documents establishing this trend on the prompting of the general public.  The first ten ammendments to our fine constitution, bundled as the @CAPS1 of @CAP

In [ ]:


#demo_data = torch.utils.data.TensorDataset(input_ids, attention_masks, np.array(essaydf['rater1_domain1'].values[0]))

Show the result of prediction of Bert and the actual result

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
max_len = 512
input_ids, attention_masks, text_len = tokenize_sentences(essaydf['essay'].tolist(), max_len)
with torch.no_grad():
  x_batch = input_ids.to(device)
  attention_masks = attention_masks.to(device)
  var = model.forward(x_batch, attention_masks)
  print("predicted score")
  print(var.to('cpu'))
  if(qnsNumber ==7):
    var = model2.forward(x_batch, attention_masks)
    print("predicted score multi model")
    result = var.to('cpu')
    # print(result)
    reformatted = np.zeros(test.shape[0])
    reformatted[0] =result[0,4]
    reformatted[1:] = result[0,:4]
    print(reformatted)
    print("real score")
    actualScore = getRealScore(essaydf)
    print(actualScore)
    #For multiple model the format is the following
    # overall score, trait1, trait 2, trait 3, trait 4
  else:
    print("real score")
    print(np.array(essaydf['domain1_score'].values[0]))
    
  


predicted score
tensor([[4.5628]])
real score
5.0


Predict any essay passages using bert below

In [ ]:
# Trying out for personally sourced essays
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
max_len = 512
input_ids, attention_masks, text_len = tokenize_sentences(essaydf['essay'].tolist(), max_len)
#inputting your own essay in the dictionary below
dictionaryEssay = {}
dictionaryEssay['essay'] = ["I am never paitent, but which I am it is only once and a while. Sitting down and not doing a thing drives me to be insane. When I’m doing nothing and it is really quiet. I get really bad headaches. In school for example and I’m waiting I get rigily like I tap my foot at tap my pencil and my desk. Tap ???everything things like that. I must do something or I go insane. Waiting in a doctors office, or flying on a plane for a while can get real boring. When I wait to long I feel lazy and tired because I’m not moving at all. I get buggy when I’m paitent, and waiting for something. It is usually no longer than @NUM1 min (half an hour) before I get real bad. I can’t be paitent very long or I get buggy. One way I help myself is playing sports and doing things constantive."]
print(dictionaryEssay)
dfown = pd.DataFrame.from_dict(dictionaryEssay)
input_ids, attention_masks, text_len = tokenize_sentences(dfown['essay'].tolist(), max_len)
with torch.no_grad():
  x_batch = input_ids.to(device)
  attention_masks = attention_masks.to(device)
  var = model.forward(x_batch, attention_masks)
  print("predicted score")
  print(var.to('cpu'))
  #For multiple model the format is the following
  # overall score, trait1, trait 2, trait 3, trait 4
  if(qnsNumber ==7):
    var = model2.forward(x_batch, attention_masks)
    print("predicted score multi model")
    print(var.to('cpu'))

{'essay': ['I am never paitent, but which I am it is only once and a while. Sitting down and not doing a thing drives me to be insane. When I’m doing nothing and it is really quiet. I get really bad headaches. In school for example and I’m waiting I get rigily like I tap my foot at tap my pencil and my desk. Tap ???everything things like that. I must do something or I go insane. Waiting in a doctors office, or flying on a plane for a while can get real boring. When I wait to long I feel lazy and tired because I’m not moving at all. I get buggy when I’m paitent, and waiting for something. It is usually no longer than @NUM1 min (half an hour) before I get real bad. I can’t be paitent very long or I get buggy. One way I help myself is playing sports and doing things constantive.']}
predicted score
tensor([[2.3319]])
